In [1]:
import diagonal_b6 as b6
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
w = b6.connect_insecure("0.0.0.0:8002")

In [3]:
name = "diagonal.works/edinburgh/school-crossing-analyis"

We use _#amenity=school_ as it has details like "name".

In [4]:
schools = w( b6.find(b6.tagged("#amenity", "school")).map(lambda f: f.get_string("name")) )

In [34]:
results = []

is_crossing = b6.keyed("crossing") # .or_(b6.keyed("#highway", "crossing"))

for school, name in tqdm(schools):

    # New feature: 'is_valid()'
    crossings = w( b6.accessible_all([school], is_crossing, 500.0, {"mode": "walk"})
                  .filter(lambda x: b6.matches(x, b6.is_valid()) )
                 )

    for _, crossing in crossings:
        roads = w( b6.find(b6.within_cap( b6.find_feature(crossing), 1.0))
                 .filter(lambda v: b6.matches(v, b6.keyed("maxspeed")))
                 .map(lambda v: v.get_string("maxspeed"))
                )

        if len(roads) == 1:
            # New feature: 'get_centroid'
            distance = w( b6.distance_meters( b6.find_feature(school).get_centroid()
                                            , b6.find_feature(crossing).get_centroid()
                                            ))

            # Note: Annoyingly slow.
            kind = w( b6.find_feature(crossing) ).get_string("crossing_ref")

            results.append(
                { "Name": name
                , "Speed": roads[0][1]
                , "Distance": distance
                , "Kind": kind
                })

df = pd.DataFrame(results)
df

  0%|          | 0/161 [00:00<?, ?it/s]

,Name,Speed,Distance,Kind
0,Pirniehall Primary School,20 mph,143.883103,pelican
1,Pirniehall Primary School,20 mph,132.092958,pelican
2,Pirniehall Primary School,20 mph,250.149500,zebra
3,Pirniehall Primary School,20 mph,278.990569,
4,Pirniehall Primary School,20 mph,260.354669,zebra
...,...,...,...,...
825,Basil Paterson School,20 mph,75.941876,
826,Basil Paterson School,20 mph,349.389772,
827,Basil Paterson School,20 mph,179.320912,
828,Basil Paterson School,20 mph,132.795679,


In [44]:
min_df = df.loc[df.groupby('Name').Distance.idxmin()]

In [45]:
min_df

,Name,Speed,Distance,Kind
658,,30 mph,124.039477,puffin
540,Abbeyhill Primary School,20 mph,116.286339,
79,BSC Edinburgh,20 mph,40.567969,zebra
714,Balerno Community High School,30 mph,160.597742,
593,Balgreen Primary School,20 mph,200.893214,toucan
...,...,...,...,...
754,Trinity Primary School,20 mph,66.186247,
312,Tynecastle High School,20 mph,169.321842,
799,Victoria Primary School,20 mph,154.740587,
413,Wester Hailes High School,40 mph,178.425976,


In [47]:
min_df[ min_df.Name.str.contains("Montessori") ]

,Name,Speed,Distance,Kind
116,Edinburgh Montessori Arts School,30 mph,266.816486,


In [48]:
min_df[ (min_df["Speed"] == "30 mph") ].sort_values(["Distance"])

,Name,Speed,Distance,Kind
110,Howdenhall & St Katharine's Special School,30 mph,45.773621,
349,Prestonfield Primary School,30 mph,69.906685,
709,Kaimes School,30 mph,77.972063,puffin
370,Portobello High School,30 mph,81.616804,
635,Oaklands School,30 mph,83.693571,
480,Dunedin School,30 mph,86.091387,
32,St Andrew's First Aid,30 mph,100.266567,
96,Broomhouse Primary School,30 mph,103.224214,puffin
99,St Joseph's RC Primary School,30 mph,112.753011,puffin
328,Gracemount High School,30 mph,113.758203,puffin


In [51]:
min_df[ min_df.Name.str.contains("Portobello High School") ]

,Name,Speed,Distance,Kind
370,Portobello High School,30 mph,81.616804,
